## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

#import keys by other way
import sys 
from config import g_key
config = g_key

# Configure gmaps API key
gmaps.configure(g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/laurabrown/Desktop/DataAnalysisClassProjects/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Albany,42.6001,-73.9662,79.36,48,65,8.37,broken clouds,US
1,1,Hobyo,5.3505,48.5268,77.29,81,100,22.77,overcast clouds,SO
2,2,Chokurdakh,70.6333,147.9167,59.68,54,92,9.26,overcast clouds,RU
3,3,Rikitea,-23.1203,-134.9692,67.73,75,100,2.06,overcast clouds,PF
4,4,Mataura,-46.1927,168.8643,38.48,99,100,3.11,light rain,NZ


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Albany,42.6001,-73.9662,79.36,48,65,8.37,broken clouds,US
1,1,Hobyo,5.3505,48.5268,77.29,81,100,22.77,overcast clouds,SO
10,10,Georgetown,5.4112,100.3354,79.90,92,20,1.14,few clouds,MY
15,15,Itarema,-2.9248,-39.9167,79.50,77,10,16.51,clear sky,BR
17,17,Hilo,19.7297,-155.0900,87.28,74,40,5.75,scattered clouds,US
20,20,Pemberton,50.3164,-122.8027,83.89,38,88,5.73,overcast clouds,CA
24,24,Acajutla,13.5928,-89.8275,82.81,76,91,5.32,overcast clouds,SV
25,25,Jamestown,42.0970,-79.2353,76.59,45,0,9.22,clear sky,US
28,28,Padang,-0.9492,100.3543,75.25,82,100,4.27,overcast clouds,ID
30,30,Marzuq,14.4000,46.4667,77.11,59,100,6.58,light rain,YE


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                204
City                   204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
Country                204
dtype: int64

In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(how='any')
clean_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Albany,42.6001,-73.9662,79.36,48,65,8.37,broken clouds,US
1,1,Hobyo,5.3505,48.5268,77.29,81,100,22.77,overcast clouds,SO
10,10,Georgetown,5.4112,100.3354,79.90,92,20,1.14,few clouds,MY
15,15,Itarema,-2.9248,-39.9167,79.50,77,10,16.51,clear sky,BR
17,17,Hilo,19.7297,-155.0900,87.28,74,40,5.75,scattered clouds,US


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,79.36,broken clouds,42.6001,-73.9662,
1,Hobyo,SO,77.29,overcast clouds,5.3505,48.5268,
10,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
15,Itarema,BR,79.50,clear sky,-2.9248,-39.9167,
17,Hilo,US,87.28,scattered clouds,19.7297,-155.0900,
20,Pemberton,CA,83.89,overcast clouds,50.3164,-122.8027,
24,Acajutla,SV,82.81,overcast clouds,13.5928,-89.8275,
25,Jamestown,US,76.59,clear sky,42.0970,-79.2353,
28,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,
30,Marzuq,YE,77.11,light rain,14.4000,46.4667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
            
            
hotel_df.head(10)      

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,79.36,broken clouds,42.6001,-73.9662,
1,Hobyo,SO,77.29,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
10,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
15,Itarema,BR,79.50,clear sky,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
17,Hilo,US,87.28,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Pemberton,CA,83.89,overcast clouds,50.3164,-122.8027,Pemberton Valley Lodge
24,Acajutla,SV,82.81,overcast clouds,13.5928,-89.8275,Laureles
25,Jamestown,US,76.59,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
28,Padang,ID,75.25,overcast clouds,-0.9492,100.3543,Mercure Padang
30,Marzuq,YE,77.11,light rain,14.4000,46.4667,


In [10]:
import numpy as np

# 7. Drop the rows where there is no Hotel Name.
c = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = c.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hobyo,SO,77.29,overcast clouds,5.3505,48.5268,Halane Hotel and Restaurent
10,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
15,Itarema,BR,79.50,clear sky,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
17,Hilo,US,87.28,scattered clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
20,Pemberton,CA,83.89,overcast clouds,50.3164,-122.8027,Pemberton Valley Lodge


In [11]:
# 8a. Create the output File (CSV)

output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))